# Libraries

In [329]:
import pandas as pd
import numpy as np

# Local Models
from library.data_entry import get_data
from library.charts import *
from library.data_preprocessing import normalized_split_data,SCALER, LOOK_BACKS
from library.inference import make_prediction


from matplotlib import pyplot as plt

# Tensorflow
import tensorflow as tf

tf.get_logger().setLevel('INFO')
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Parameters

In [330]:
start_date = "2020-01-01"
end_date = "2023-05-01"

predict_date = "2023-04-10"
predict_end_date = '2023-07-10'

ticker = 'NFLX'

# features = ['Close', 'RSI', 'EMA_8', 'MACD_12_26_9', 'MACDs_12_26_9']
features = ['Close']

model_name = 'bidirectional_LSTM'

# Get Data

In [331]:
data = get_data(ticker, start_date, end_date, )
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume,RSI,EMA_8,EMA_20,Shift,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
Date,,,,,,,,,,,,
2020-02-20,386.559998,389.700012,376.679993,386.000000,4079400,72.288687,380.321469,367.544882,-0.190002,14.327105,0.861831,13.465274
2020-02-21,385.329987,387.320007,377.899994,380.070007,3930100,65.274237,380.265589,368.737751,-5.929993,13.614481,0.119366,13.495115
2020-02-24,364.760010,372.820007,361.000000,368.700012,6936400,54.425170,377.695461,368.734157,-11.369995,11.993999,-1.200893,13.194892
2020-02-25,372.000000,375.649994,357.720001,360.089996,6481200,47.957943,373.783135,367.910903,-8.610016,9.900867,-2.635220,12.536087
2020-02-26,366.309998,382.000000,365.000000,379.239990,8934100,59.442564,374.995770,368.989864,19.149994,9.675754,-2.288266,11.964021
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-24,330.200012,334.660004,326.750000,329.019989,5586600,49.465438,330.491494,330.791229,1.039978,1.246159,-1.711426,2.957584
2023-04-25,328.500000,328.660004,321.100006,322.549988,5426600,45.419038,328.726715,330.006349,-6.470001,0.452452,-2.004106,2.456558
2023-04-26,321.359985,325.899994,320.470001,321.149994,4623200,44.573700,327.042999,329.162887,-1.399994,-0.286235,-2.194234,1.907999


In [332]:
x_train, y_train, x_val, y_val = normalized_split_data(data, features)

(744, 60, 1) (744,) (160, 60, 1) (160, 1)


# Model

In [333]:
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(x_train.shape[1], 1)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=False)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(25, activation='relu'),
    tf.keras.layers.Dense(1),
])

model.summary()

2023-07-20 14:44:08.295889: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-20 14:44:08.296834: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-20 14:44:08.297469: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 60, 128)          33792     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 60, 128)           0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 25)                1625      
                                                                 
 dense_1 (Dense)             (None, 1)                 2

2023-07-20 14:44:08.498334: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-20 14:44:08.499236: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_1_grad/concat/split_1/split_dim' with dtype int32
	 [[{{node gradients/split_1_grad/concat/split_1/split_dim}}]]
2023-07-20 14:44:08.575512: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [334]:
tf.keras.backend.clear_session()

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Тrain the model
history = model.fit(x_train, y_train,
                    # validation_data= [x_val, y_val],
                    batch_size=1,
                    epochs=11,
                    use_multiprocessing=True)

model.save(f'./model/{model_name}.h5')

Epoch 1/11


2023-07-20 14:44:08.907450: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-20 14:44:08.909171: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-20 14:44:08.910280: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

744/744 [==============================] - 13s 14ms/step - loss: 0.0089 - accuracy: 0.0027
Epoch 2/11
744/744 [==============================] - 10s 14ms/step - loss: 0.0041 - accuracy: 0.0027
Epoch 3/11
744/744 [==============================] - 10s 14ms/step - loss: 0.0035 - accuracy: 0.0027
Epoch 4/11
744/744 [==============================] - 10s 14ms/step - loss: 0.0026 - accuracy: 0.0027
Epoch 5/11
744/744 [==============================] - 10s 14ms/step - loss: 0.0027 - accuracy: 0.0027
Epoch 6/11
744/744 [==============================] - 10s 14ms/step - loss: 0.0026 - accuracy: 0.0027
Epoch 7/11
744/744 [==============================] - 10s 14ms/step - loss: 0.0023 - accuracy: 0.0027
Epoch 8/11
744/744 [==============================] - 10s 14ms/step - loss: 0.0023 - accuracy: 0.0027
Epoch 9/11
744/744 [==============================] - 10s 14ms/step - loss: 0.0023 - accuracy: 0.0027
Epoch 10/11
744/744 [==============================] - 10s 14ms/step - loss: 0.0020 - accurac

# Model Evalutaion

In [335]:
predictions = model.predict(x_val)
predictions = SCALER.inverse_transform(predictions)

# For finding error we use RMSE formula, but MSE can be used too
rmse = np.sqrt(np.mean(((predictions - y_val) ** 2)))
print(f'RMSE {model_name}: {rmse}')

2023-07-20 14:46:03.407105: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-20 14:46:03.408249: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-20 14:46:03.409122: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

5/5 [==============================] - 1s 7ms/step
RMSE bidirectional_LSTM: 10.544238133578556


# Validation on unseen Data

In [336]:
inference_model = tf.keras.models.load_model(f'./model/{model_name}.h5')

test_data = get_data(ticker, start_date=predict_date , end_date=predict_end_date, is_raw=True)
result = make_prediction(test_data, model)

original = get_data(ticker, start_date=predict_date, end_date=str(result.iloc[-1,0])[0:10], is_raw=True)
original = original.reset_index()
original = original.filter(['Date', 'Close'])

prediction_chart(prediction_data=result, original_data= original)

2023-07-20 14:46:04.207124: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-20 14:46:04.207953: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-20 14:46:04.208591: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

[*********************100%***********************]  1 of 1 completed

2023-07-20 14:46:04.480266: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis' with dtype int32 and shape [1]
	 [[{{node gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis}}]]
2023-07-20 14:46:04.503553: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-20 14:46:04.504222: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message):

2023-07-20 14:46:04.816208: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-20 14:46:04.817386: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-20 14:46:04.818478: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 0s 15ms/step
[*********************100%***********************]  1 of 1 completed
